<a href="https://colab.research.google.com/github/wwoskie/Biostat_BI_2022/blob/biostat_project_1/biostat_project_1/biostat_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Данные разбиты на несколько файлов. Нам надо как-то объединить наблюдения в единую таблицу. Пожалуйста, напишите пользовательскую функцию, благодаря которой мы сможем собрать все наши наблюдения в одну таблицу. Так как олимпиады будут проводиться регулярно, то функция должна объединять все файлы определенного расширения из заданной папки (тип расширения передается как аргумент функции). (5 баллов)

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
def table_merger(dir_name, file_extension, save_table = False, sep = ',', sheet = 0):
    
    def pd_read_csv_(path, sep, sheet):
        return pd.read_csv(path, sep = sep)
    def pf_read_excel_(path, sep, sheet):
        return pd.read_excel(path, sheet_name = sheet)
    
    reading_functions_dct = {
        pd_read_csv_: ['.csv', '.tsv', '.txt'], # я думаю, что все распространенные текстовые форматы сможет прочитать read_csv, только с соответствующими разделителями
        pf_read_excel_: ['.xlsx'] # для всего остального уже нет мастеркард(, но есть excel
    }
    
    reading_ext_dct = {} # выворачиваю словарь, чтобы обращаться по расширению
    for key, item in reading_functions_dct.items():
        for ext in item:
            reading_ext_dct[ext] = key
            
    
    files_to_read = [file for file in os.listdir(dir_name) if file.endswith(file_extension)]
    
    if len(files_to_read) > 0:
        df = reading_ext_dct[file_extension](path = os.path.join(dir_name, files_to_read[0]), sep = sep, sheet = sheet)
    else:
        df = None
        
    if len(files_to_read) > 1:
        for file in files_to_read[1:]:
            df = pd.concat([df, reading_ext_dct[file_extension](path = os.path.join(dir_name, file), sep = sep, sheet = sheet)])
            
    if save_table:
        df.to_csv(os.path.join(dir_name, f'merged_{files_to_read[0]}_{files_to_read[-1]}'), index=False)
    
    return df

In [3]:
df_ath = table_merger(dir_name = 'athlete_events', file_extension = '.csv')

2. Посмотрите, действительно ли все данные корректны? Если найдете, что что-то не так, то исправьте это, пожалуйста. Объясните, почему вы воспользовались именно этим подходом. Может быть у него есть альтернативы? (5 баллов)

In [4]:
df_ath

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,124517,William Van Dijck,M,23.0,185.0,65.0,Belgium,BEL,1984 Summer,1984.0,Summer,Los Angeles,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN
1,124517,William Van Dijck,M,27.0,185.0,65.0,Belgium,BEL,1988 Summer,1988.0,Summer,Seoul,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN
2,124517,William Van Dijck,M,31.0,185.0,65.0,Belgium,BEL,1992 Summer,1992.0,Summer,Barcelona,Athletics,"Athletics Men's 3,000 metres Steeplechase",NaN
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold
4,124519,"Everdina ""Edith"" van Dijk",F,35.0,176.0,66.0,Netherlands,NED,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 10 kilometres Open Water,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22831,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Mixed Doubles,NaN
22832,12004,Peter Grant Blackburn,M,28.0,179.0,78.0,Australia,AUS,1996 Summer,1996.0,Summer,Atlanta,Badminton,Badminton Men's Doubles,NaN
22833,12004,Peter Grant Blackburn,M,32.0,179.0,78.0,Australia,AUS,2000 Summer,2000.0,Summer,Sydney,Badminton,Badminton Mixed Doubles,NaN
22834,12004,Peter Grant Blackburn,M,32.0,179.0,78.0,Australia,AUS,2000 Summer,2000.0,Summer,Sydney,Badminton,Badminton Men's Doubles,NaN


Так-с, ну в последней строке часть данных ушла в отпуск. Наверное, такие строки можно будет выявить по большому числу нанов, мне они не нравятся, я бы их выкинул

In [5]:
df_ath.apply(lambda x: x.count(), axis=1).value_counts()

14    177105
12     43633
15     30181
13     12459
11      7729
7          2
2          1
6          1
10         1
1          1
8          1
9          1
dtype: int64

Тут видно, что большая часть наблюдений содержит как минимум 11 записей в строке. Те, у кого 15 записей выиграли медали, они молодцы, те, у кого 14 записей тоже молодцы, так как съездили на олимпийские игры, но, скорее всего, медалей не привезли. Вопросы есть к тем, у кого записей меньше, почему...

In [6]:
df_ath[df_ath.apply(lambda x: x.count(), axis=1) < 11]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
22409,113716,Pietro Spec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22586,23433,Hadj Moussa Coulibaly,M,23.0,NaN,NaN,Mali,MLI,2004 Summe,NaN,NaN,NaN,NaN,NaN,NaN
22557,34727,Carlotta Ferlito,F,17.0,160.0,50.0,Italy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22642,68370,Helmut Lehmann,M,25.0,178.0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22781,91137,Georgios Pantos,M,NaN,NaN,NaN,Athens-2,GRE,1906 Summer,1906.0,Summer,Athina,Footba,NaN,NaN
22463,124516,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22659,45919,Yuka Harada,F,28.0,170.0,60.0,Japan,JP,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22835,12005,Andrea Melissa Blackett,F,24.0,167.0,59.0,Barbados,BAR,2000 Su,NaN,NaN,NaN,NaN,NaN,NaN


Видим, что этих ребят уже не спасти, наверное, с ними придется расстаться(

In [7]:
df_ath = df_ath[df_ath.apply(lambda x: x.count(), axis=1) > 10]

Теперь попробуем пойти по колонкам и посмотреть, что тут у нас с заполненностью данными

In [8]:
for col in df_ath.columns:
    print(col, sum(pd.isnull(df_ath[col])))

ID 0
Name 0
Sex 0
Age 9473
Height 60168
Weight 62872
Team 0
NOC 0
Games 0
Year 0
Season 0
City 0
Sport 0
Event 0
Medal 231325


Больше всего пострадали медали (что логично, их мы трогать не будет), а также вес, рост и возраст. Возраст и рост можно попробовать воостановить по другим записям о спортсмене, вес тоже можно предположить и взять как среднее, например. Радует, что в айди нет пропущенных значений, но данные на всякий случай просмотреть, вдруг там что-то криво заполнено

In [9]:
no_age = set(df_ath[pd.isnull(df_ath['Age'])]['ID']) # только уникальные айди
len(set(no_age)) # без возраста вот столько вот уникальных спортсменов, попробуем его посчитать

6367

In [10]:
df_no_age = df_ath[df_ath['ID'].isin(no_age)] # в этот датафрейм должны попасть все спортсмены с айди без возраста. Вдруг для них есть другая запись
df_no_age

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
52,124544,Charles Van Doorselaer,M,NaN,NaN,NaN,Belgium,BEL,1920 Summer,1920.0,Summer,Antwerpen,Cycling,"Cycling Men's Team Pursuit, 4,000 metres",NaN
61,124550,R. Van Dorpe,M,NaN,NaN,NaN,Belgium,BEL,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Architecture, Unknown E...",NaN
79,124561,Herman Van Duyzen,M,NaN,NaN,NaN,Belgium,BEL,1924 Summer,1924.0,Summer,Paris,Wrestling,"Wrestling Men's Lightweight, Freestyle",NaN
182,124616,Louis Van Gheem,M,NaN,NaN,NaN,Belgium,BEL,1928 Summer,1928.0,Summer,Amsterdam,Water Polo,Water Polo Men's Water Polo,NaN
216,124627,Jean Van Guysse,M,NaN,NaN,NaN,Belgium,BEL,1908 Summer,1908.0,Summer,London,Gymnastics,Gymnastics Men's Individual All-Around,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22815,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Horse Vault,NaN
22816,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Parallel Bars,NaN
22817,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Horizontal Bar,NaN
22818,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Rings,NaN


In [11]:
col = 'Age'
print(col, sum(pd.isnull(df_no_age[col])))

Age 9473


К сожалению, это число совпадает с тем, что было получено ранее для возраста, поэтому восстановить возраст не получится. Грустно(
Выбрасывать я их не буду, они могут пригодиться в других местах, ведь не везде придется считать что-то с возрастом
Проделаем аналогичные операции для роста

In [12]:
no_height = set(df_ath[pd.isnull(df_ath['Height'])]['ID']) # только уникальные айди
len(set(no_height)) # 

33914

In [13]:
df_no_height = df_ath[df_ath['ID'].isin(no_height)]
df_no_height

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold
10,124521,Hilbert Van Dijk,M,38.0,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Fencing,"Fencing Men's epee, Team",NaN
14,124525,Johannes Wilhelmus Maria van Dijk,M,31.0,NaN,NaN,Minerva Amsterdam,NED,1900 Summer,1900.0,Summer,Paris,Rowing,Rowing Men's Coxed Eights,Bronze
16,124527,"Cornelis ""Nelis"" van Dijk",M,16.0,NaN,NaN,Netherlands,NED,1920 Summer,1920.0,Summer,Antwerpen,Boxing,Boxing Men's Flyweight,NaN
34,124534,"Gustaaf Lodewijk Gerhard ""Guus"" van Ditzhuyzen",M,19.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Rowing,Rowing Men's Coxed Eights,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22818,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Rings,NaN
22819,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Pommelled Horse,NaN
22820,12000,Jerald Clifford Blackburn,M,20.0,NaN,NaN,Canada,CAN,1948 Summer,1948.0,Summer,London,Boxing,Boxing Men's Welterweight,NaN
22826,12002,Mackenzie Blackburn,M,21.0,NaN,NaN,Chinese Taipei,TPE,2014 Winter,2014.0,Winter,Sochi,Short Track Speed Skating,Short Track Speed Skating Men's 500 metres,NaN


In [14]:
col = 'Height'
print(col, sum(pd.isnull(df_no_height[col])))

Height 60168


Вот незадача! Тут тоже ничего не вышло, а ведь так хотелось... (ну либо я делаю что-то не так)

In [15]:
no_weight = set(df_ath[pd.isnull(df_ath['Weight'])]['ID']) # только уникальные айди
len(set(no_weight)) # 

34883

In [16]:
df_no_weight = df_ath[df_ath['ID'].isin(no_weight)]
df_no_weight

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
3,124518,"Daniel ""Daan"" van Dijk",M,21.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Cycling,"Cycling Men's Tandem Sprint, 2,000 metres",Gold
10,124521,Hilbert Van Dijk,M,38.0,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Fencing,"Fencing Men's epee, Team",NaN
14,124525,Johannes Wilhelmus Maria van Dijk,M,31.0,NaN,NaN,Minerva Amsterdam,NED,1900 Summer,1900.0,Summer,Paris,Rowing,Rowing Men's Coxed Eights,Bronze
16,124527,"Cornelis ""Nelis"" van Dijk",M,16.0,NaN,NaN,Netherlands,NED,1920 Summer,1920.0,Summer,Antwerpen,Boxing,Boxing Men's Flyweight,NaN
34,124534,"Gustaaf Lodewijk Gerhard ""Guus"" van Ditzhuyzen",M,19.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Rowing,Rowing Men's Coxed Eights,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22818,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Rings,NaN
22819,11999,Brian Reginald Blackburn,M,NaN,NaN,NaN,Australia,AUS,1956 Summer,1956.0,Summer,Melbourne,Gymnastics,Gymnastics Men's Pommelled Horse,NaN
22820,12000,Jerald Clifford Blackburn,M,20.0,NaN,NaN,Canada,CAN,1948 Summer,1948.0,Summer,London,Boxing,Boxing Men's Welterweight,NaN
22826,12002,Mackenzie Blackburn,M,21.0,NaN,NaN,Chinese Taipei,TPE,2014 Winter,2014.0,Winter,Sochi,Short Track Speed Skating,Short Track Speed Skating Men's 500 metres,NaN


In [17]:
col = 'Weight'
print(col, sum(pd.isnull(df_no_weight[col])))

Weight 62872


Чуда не случилось( Возможно, в некоторых олимпиадах спортсменов еще не было принято измерять или кто-то был в отпуске. Теперь посмотрим на валидность самих данных, вдруг тут кто-то что-то напутал при заполнении. Начнем с пола

In [18]:
df_ath['Sex'].value_counts() # тут появился третий пол

M    196585
F     74520
G         2
Name: Sex, dtype: int64

In [19]:
df_ath.loc[df_ath['Sex'] == 'G']

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
42,79609,Pavel Mike,G,22.0,182.0,79.0,Czechoslovakia,TCH,1972 Summer,1972.0,Summer,Munich,Handball,Handball Men's Handball,Silver
74,79630,Anatoly Mikhaylin,G,37.0,NaN,NaN,Russia,RUS,1996 Summer,1996.0,Summer,Atlanta,Sailing,Sailing Mixed Two Person Keelboat,NaN


Погуглил этих спортсменов. Скорее всего это просто ошибка при заполнении и они оба мужчины
https://wiki2.org/en/Pavel_Mike%C5%A1
https://en.wikipedia.org/wiki/Anatoly_Mikhaylin

In [20]:
df_ath.loc[df_ath['Sex'] == 'G', 'Sex'] = 'M' # исправим это в данных

In [21]:
df_ath['Sex'].value_counts()

M    196587
F     74520
Name: Sex, dtype: int64

Теперь посмотрим на возраст, в чате ходили слухи про детей и стариков, проверим правдивы ли они

In [22]:
df_ath['Age'].min(), df_ath['Age'].max()

(10.0, 240.0)

Ого, и правда, надо бы что-то с этим сделать. Для начала поймем, какие там вообще границы

In [23]:
all_ages = np.sort(df_ath['Age'][df_ath['Age'].notnull()].unique())
all_ages

array([ 10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,
        21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,
        32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,
        43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  52.,  53.,
        54.,  55.,  56.,  57.,  58.,  59.,  60.,  61.,  62.,  63.,  64.,
        65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  73.,  74.,  75.,
        76.,  77.,  80.,  81.,  84.,  88.,  96.,  97., 240.])

Скажем, 10 лет еще выглядят более-менее правдоподобно, но вот 240...

In [24]:
df_ath.loc[df_ath['Age'] == 240]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
38,23459,Flicien Jules mile Courbet,M,240.0,NaN,NaN,Belgium,BEL,1912 Summer,1912.0,Summer,Stockholm,Swimming,Swimming Men's 200 metres Breaststroke,NaN


Видимо, тут лишний ноль и спортсмену было 24 года, потому что про него есть статья на википедии (род. 1888)
https://en.wikipedia.org/wiki/F%C3%A9licien_Courbet

In [25]:
df_ath.loc[df_ath['Age'] == 240, 'Age'] = 24 # исправим это в данных

In [26]:
all_ages = np.sort(df_ath['Age'][df_ath['Age'].notnull()].unique())
all_ages

array([10., 11., 12., 13., 14., 15., 16., 17., 18., 19., 20., 21., 22.,
       23., 24., 25., 26., 27., 28., 29., 30., 31., 32., 33., 34., 35.,
       36., 37., 38., 39., 40., 41., 42., 43., 44., 45., 46., 47., 48.,
       49., 50., 51., 52., 53., 54., 55., 56., 57., 58., 59., 60., 61.,
       62., 63., 64., 65., 66., 67., 68., 69., 70., 71., 72., 73., 74.,
       75., 76., 77., 80., 81., 84., 88., 96., 97.])

Взглянем на молодых спортсменов. Данные как будто выглядят правдоподобно, большинство представлено в тех видах спорта, где бывают молодые спортсмены.

In [29]:
df_ath[df_ath['Age'].isin(all_ages[:3])]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
1078,125092,tienne Nol Henri Vandernotte,M,12.0,NaN,37.0,France,FRA,1936 Summer,1936.0,Summer,Berlin,Rowing,Rowing Men's Coxed Pairs,Bronze
1079,125092,tienne Nol Henri Vandernotte,M,12.0,NaN,37.0,France,FRA,1936 Summer,1936.0,Summer,Berlin,Rowing,Rowing Men's Coxed Fours,Bronze
2770,125944,Ines Vercesi,F,12.0,NaN,NaN,Italy,ITA,1928 Summer,1928.0,Summer,Amsterdam,Gymnastics,Gymnastics Women's Team All-Around,Silver
3505,126307,Liana Vicens,F,11.0,158.0,50.0,Puerto Rico,PUR,1968 Summer,1968.0,Summer,Mexico City,Swimming,Swimming Women's 100 metres Breaststroke,NaN
3506,126307,Liana Vicens,F,11.0,158.0,50.0,Puerto Rico,PUR,1968 Summer,1968.0,Summer,Mexico City,Swimming,Swimming Women's 200 metres Breaststroke,NaN
3507,126307,Liana Vicens,F,11.0,158.0,50.0,Puerto Rico,PUR,1968 Summer,1968.0,Summer,Mexico City,Swimming,Swimming Women's 200 metres Individual Medley,NaN
3508,126307,Liana Vicens,F,11.0,158.0,50.0,Puerto Rico,PUR,1968 Summer,1968.0,Summer,Mexico City,Swimming,Swimming Women's 4 x 100 metres Medley Relay,NaN
4937,127018,Yelena Germanovna Vodorezova (-Buyanova),F,12.0,161.0,47.0,Soviet Union,URS,1976 Winter,1976.0,Winter,Innsbruck,Figure Skating,Figure Skating Women's Singles,NaN
22168,113580,Inge Srensen (-Tabur),F,12.0,NaN,NaN,Denmark,DEN,1936 Summer,1936.0,Summer,Berlin,Swimming,Swimming Women's 200 metres Breaststroke,Bronze
20632,22411,Magdalena Cecilia Colledge,F,11.0,152.0,NaN,Great Britain,GBR,1932 Winter,1932.0,Winter,Lake Placid,Figure Skating,Figure Skating Women's Singles,NaN


Теперь посмотрим на возрастных. Ого, не знал, что были олимпийские игры по рисованию! В целом, данные тоже выглядят правдоподобно.

In [30]:
df_ath[df_ath['Age'].isin(all_ages[all_ages > 64])]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
892,124990,"Cornelis Anthony ""Kees"" van Waning",M,66.0,NaN,NaN,Netherlands,NED,1928 Summer,1928.0,Summer,Amsterdam,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
1989,125573,Daniel Vzquez Daz,M,66.0,NaN,NaN,Spain,ESP,1948 Summer,1948.0,Summer,London,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
3775,126447,Johann Vierthaler,M,67.0,NaN,NaN,Germany,GER,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Sculpturing, Unknown Event",NaN
4574,126841,Konstantns Visotskis,M,68.0,NaN,NaN,Latvia,LAT,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
4575,126841,Konstantns Visotskis,M,68.0,NaN,NaN,Latvia,LAT,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21360,11270,"Lisa ""Louise"" Bianchini (Wadstrm-)",F,68.0,NaN,NaN,Sweden,SWE,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
21361,11270,"Lisa ""Louise"" Bianchini (Wadstrm-)",F,68.0,NaN,NaN,Sweden,SWE,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
21362,11270,"Lisa ""Louise"" Bianchini (Wadstrm-)",F,68.0,NaN,NaN,Sweden,SWE,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN
21363,11270,"Lisa ""Louise"" Bianchini (Wadstrm-)",F,68.0,NaN,NaN,Sweden,SWE,1936 Summer,1936.0,Summer,Berlin,Art Competitions,"Art Competitions Mixed Painting, Paintings",NaN


In [31]:
df_ath[df_ath['Age'].isin(all_ages[-3:])]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
8329,128719,John Quincy Adams Ward,M,97.0,NaN,NaN,United States,USA,1928 Summer,1928.0,Summer,Amsterdam,Art Competitions,"Art Competitions Mixed Sculpturing, Statues",NaN
15438,31173,Thomas Cowperthwait Eakins,M,88.0,NaN,NaN,United States,USA,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
15439,31173,Thomas Cowperthwait Eakins,M,88.0,NaN,NaN,United States,USA,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
15440,31173,Thomas Cowperthwait Eakins,M,88.0,NaN,NaN,United States,USA,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN
7477,49663,Winslow Homer,M,96.0,NaN,NaN,United States,USA,1932 Summer,1932.0,Summer,Los Angeles,Art Competitions,"Art Competitions Mixed Painting, Unknown Event",NaN


С полом и возрастом разобрались, теперь взглянем на рост

In [32]:
all_height = np.sort(df_ath['Height'][df_ath['Height'].notnull()].unique())
all_height

array([127., 128., 130., 131., 132., 133., 135., 136., 137., 138., 139.,
       140., 141., 142., 143., 144., 145., 146., 147., 148., 149., 150.,
       151., 152., 153., 154., 155., 156., 157., 158., 159., 160., 161.,
       162., 163., 164., 165., 166., 167., 168., 169., 170., 171., 172.,
       173., 174., 175., 176., 177., 178., 179., 180., 181., 182., 183.,
       184., 185., 186., 187., 188., 189., 190., 191., 192., 193., 194.,
       195., 196., 197., 198., 199., 200., 201., 202., 203., 204., 205.,
       206., 207., 208., 209., 210., 211., 212., 213., 214., 215., 216.,
       217., 218., 219., 220., 221., 223., 226., 340.])

Рост в почти 3.5 метра выглядит не ок, взглянем

In [34]:
df_ath.loc[df_ath['Height'] == 340]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
227,23549,Kirsty Leigh Coventry (-Seward),F,28.0,340.0,64.0,Zimbabwe,ZIM,2012 Summer,2012.0,Summer,London,Swimming,Swimming Women's 200 metres Individual Medley,NaN


In [35]:
df_ath.loc[df_ath['ID'] == 23549]

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
214,23549,Kirsty Leigh Coventry (-Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 50 metres Freestyle,NaN
215,23549,Kirsty Leigh Coventry (-Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 100 metres Freestyle,NaN
216,23549,Kirsty Leigh Coventry (-Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 100 metres Backstroke,NaN
217,23549,Kirsty Leigh Coventry (-Seward),F,16.0,176.0,64.0,Zimbabwe,ZIM,2000 Summer,2000.0,Summer,Sydney,Swimming,Swimming Women's 200 metres Individual Medley,NaN
218,23549,Kirsty Leigh Coventry (-Seward),F,20.0,176.0,64.0,Zimbabwe,ZIM,2004 Summer,2004.0,Summer,Athina,Swimming,Swimming Women's 100 metres Backstroke,Silver
219,23549,Kirsty Leigh Coventry (-Seward),F,20.0,176.0,64.0,Zimbabwe,ZIM,2004 Summer,2004.0,Summer,Athina,Swimming,Swimming Women's 200 metres Backstroke,Gold
220,23549,Kirsty Leigh Coventry (-Seward),F,20.0,176.0,64.0,Zimbabwe,ZIM,2004 Summer,2004.0,Summer,Athina,Swimming,Swimming Women's 200 metres Individual Medley,Bronze
221,23549,Kirsty Leigh Coventry (-Seward),F,24.0,176.0,64.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 100 metres Backstroke,Silver
222,23549,Kirsty Leigh Coventry (-Seward),F,24.0,176.0,64.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 200 metres Backstroke,Gold
223,23549,Kirsty Leigh Coventry (-Seward),F,24.0,176.0,64.0,Zimbabwe,ZIM,2008 Summer,2008.0,Summer,Beijing,Swimming,Swimming Women's 200 metres Individual Medley,Silver


Завидная стабильность в весе! Про рост такого не скажешь, поэтому заменим

In [37]:
df_ath.loc[df_ath['Height'] == 340, 'Height'] = 176

In [38]:
df_ath.groupby('ID').agg(mean_height = ('Height', 'mean'))

,mean_height
ID,
1,180.0
2,170.0
3,NaN
4,NaN
5,185.0
...,...
135567,183.0
135568,171.0
135569,179.0
